# Bayesian preprocessing? (Rename this as needed)

Testing a renewal model with a Bayesian inference of parameters for preprocessing and estimating R(t).

In [1]:
import glob
import importlib
import os
from collections import defaultdict
from pathlib import Path
import re
import sys 

import epiweeks
import numpy as np
import pandas as pd
import plotly 
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots
import scipy
from plotly.subplots import make_subplots

sys.path.append("../../../Rtrend_project/")
if os.path.basename(os.getcwd()) != "programs":
    os.chdir("..")  # Move to project root
print(f"Working directory: {os.getcwd()}")

Working directory: /Users/pventura/NonCloudStorage/Flu_forecast/programs


In [92]:
# Example definition of parameters
# ================================
POP_RATE_REF = int(1E5)  # Reference constant for calculating rates. Eg: hospitalizations *per 100k people* 
WEEKLEN = 7  # Constant: number of days in a week

initial_model_params = dict(
    # Model params
    pop=1E6,  # State population
    c_pop_init=45,  # c(0) = c_pop_init * pop / 1E5. Initial rate of daily infection incidence
    t_init=pd.Timestamp("2024-01-01"),
    t_present=pd.Timestamp("2024-01-01") + pd.Timedelta("7w"),
    roi_len=56,  # Number of days to simulate the model for.
    
    # Reproduction number model
    r_bias=0.00,
    drift_pop_coef=0.1E-5,  # drift = coef * 1E5 / pop * c(t)
    r_init=1.15,
    
    # Generation time
    tg_gamma_shape=10.,
    tg_gamma_rate=3.333333,
    tg_max=15,
    
    # Infection-to-hospitalization assigned report 
    hosp_delay_gamma_shape=25.,
    hosp_delay_gamma_rate=5.0,
    hosp_delay_max=15,
    hosp_fraction=0.01,
    likelihood_overdisp=0.0002,
)

mcmc_param_names = [
    "r_bias", 
    "r_init", 
    "c_pop_init", 
    "likelihood_overdisp"
]
mcmc_param_sigmas = dict(
    r_bias=0.0005,
    r_init=0.02,
    c_pop_init=0.5,
    likelihood_overdisp=0.01,
)
mcmc_param_bounds = dict(
    r_bias=(-np.inf, np.inf),
    r_init=(0, np.inf),
    c_pop_init=(0.01, POP_RATE_REF),
    likelihood_overdisp=(0.000000001, np.inf),
)

# ==============

class ModelParamsMain:
    c_pop_init: float
    pop: int
    r_bias: float
    drift_pop_coef: float
    r_init: float
    t_init: pd.Timestamp  # Date of the first simulated day. Start of the preprocessing ROI.
    roi_len: int
    tg_gamma_shape: float
    tg_gamma_rate: float
    tg_max: float
    hosp_delay_gamma_shape: float
    hosp_delay_gamma_rate: float
    hosp_delay_max: int
    hosp_fraction: float
    likelihood_overdisp: float

def run_model_once(**kwargs):
    p = ModelParamsMain()
    
    # Require all parameters created as annotations (type hints)
    for name in ModelParamsMain.__annotations__:
        setattr(p, name, kwargs[name])
        
    # Build the generation time distribution
    # ==================
    # CONVENTION: tg_pmf(s) = probab. of tg being of `s` days = CDF(s+1) - CDF(s)
    s_array = np.arange(1, p.tg_max)  # As of in Rtrend library, tg_max itself is also excluded.
    tg_gamma = scipy.stats.gamma(a=p.tg_gamma_shape, scale=1. / p.tg_gamma_rate)  # The continuous-time gamma that creates the TG PMF
    tg_pmf_array = np.diff(tg_gamma.cdf(s_array), append=tg_gamma.cdf(p.tg_max))  # This excludes the 0-1 day period
    tg_pmf_sr = pd.Series(tg_pmf_array, index=s_array) / tg_pmf_array.sum()
    
    
    # Renewal + Reproduction number model
    # =================
    
    # Burn-in period, just simulate a...
    # ----------------------------
    # -()- constant incidence
    ct_burnin_index = pd.date_range(start=p.t_init - pd.Timedelta(p.tg_max, "D"), end=p.t_init - pd.Timedelta("1d"), freq="D")  # Stops one day before simulation period
    # c_init = int(round(p.c_pop_init * p.pop / POP_RATE_REF))
    c_init = float(p.c_pop_init * p.pop / POP_RATE_REF)
    ct_burnin_sr = pd.Series(c_init, index=ct_burnin_index)
    
    # -()- constant reproduction number
    # pass
    
    # Main ROI: Simulate a renewal model AND R(t) with drift, from t0 to t0 + roi_len
    # --------------------------
    ct_simulate_index = pd.date_range(start=p.t_init, end=p.t_init + pd.Timedelta(p.roi_len, "D"), freq="D")
    ct_simulate_sr = pd.Series(np.nan, index=ct_simulate_index)
    ct_sr = pd.concat([ct_burnin_sr, ct_simulate_sr])  # Incidence at all periods
    ct_current = ct_burnin_sr.iloc[-1]
    
    rt_sr = pd.Series(np.nan, index=ct_simulate_index)
    rt_current = p.r_init
    
    drift_coef = p.drift_pop_coef * POP_RATE_REF / p.pop
    
    for i_t, t in enumerate(ct_simulate_index):
        
        # Reproduction number
        rt_next = rt_current * np.exp(p.r_bias - drift_coef * ct_current)
        
        # Total infectiousness (convolution of past incidence with generation time)
        tot_infec = 0.
        for i_s, s in enumerate(tg_pmf_sr.index):
            tot_infec += ct_sr.loc[t - pd.Timedelta(s, "D")] * tg_pmf_sr[s]
        
        # Incidence of the next step
        ct_next = tot_infec * rt_next
        ct_sr.loc[t] = ct_next
        rt_sr.loc[t] = rt_next
        
        rt_current = rt_next
        ct_current = ct_next
            
    return ct_sr, rt_sr


def calculate_loglikelihood(ct_sr: pd.Series, rt_sr: pd.Series, data_sr: pd.Series, weekday="Saturday", **kwargs):
    p = ModelParamsMain()
    
    # Require all parameters created as annotations (type hints)
    for name in ModelParamsMain.__annotations__:
        setattr(p, name, kwargs[name])
            
    # =======
        
    # # Check whether dates in the data are Saturdays
    # if not (data_sr.index.day_name() == weekday).all():
    #     raise ValueError(f"Data should be `weekday`= {weekday}")
    # 
    # # Check whether all dates in the data are in the model series
    # if not data_sr.index.isin(ct_sr.index).all():
    #     raise ValueError(f"All dates in data should be contained in the model series")

    # =========
    
    # Build the infection-to-hospitalization delay distribution
    # ==================
    # CONVENTION: hosp_pmf(s) = probab. of an infection at day 0 being hospitalized at day s = CDF(s+1) - CDF(s)
    s_array = np.arange(1, p.hosp_delay_max)  # As of in Rtrend library, tg_max itself is also excluded.
    hosp_gamma = scipy.stats.gamma(a=p.hosp_delay_gamma_shape, scale=1. / p.hosp_delay_gamma_rate)  # The continuous-time gamma that creates the TG PMF
    hosp_pmf_array = np.diff(hosp_gamma.cdf(s_array), append=hosp_gamma.cdf(p.hosp_delay_max))  # For simplicity, it also excludes the 0-1 day period
    hosp_pmf_sr = pd.Series(hosp_pmf_array, index=s_array) / hosp_pmf_array.sum()
    
    # --- Process and Aggregate model outputs
    # ==================
    # fit_window_daily_index = pd.DatetimeIndex(
    #     [day for week_end_date in data_sr.index for day in pd.date_range(week_end_date - pd.Timedelta(days=6), week_end_date, freq="D")]
    # )
    
    # --- Calculate the weekly series of _expectancy_ (average) of hospitalizations from the model
    # For each week of data...
    mu_sr = pd.Series(np.nan, index=data_sr.index, name="expectancy_hosp")  # Expectancy of weekly hospitalizations, according to the model
    for i_w, w in enumerate(data_sr.index):
        w: pd.Timestamp
        mu_w = 0.  # Expectancy of weekly hospitalizations
        week_daily_index = pd.date_range(end=w, freq="D", periods=WEEKLEN)
        
        # For each day in the week...
        for i_t, t in enumerate(week_daily_index):
            # For each past lag `s` in days...
            for i_s, s in enumerate(hosp_pmf_sr.index):
                # c(t - s) * f_h * eta(s)
                mu_w += p.hosp_fraction * ct_sr.loc[t - pd.Timedelta(s, "D")] * hosp_pmf_sr.loc[s]
                
        mu_sr.loc[w] = mu_w
        
    # --- Calculate the overall loglikelihood
    n = 1 / p.likelihood_overdisp
    p_sr = n / (n + mu_sr)  # 
    ll_sr = scipy.stats.nbinom.logpmf(k=data_sr, n=n, p=p_sr)
    total_ll = ll_sr.sum()

    return mu_sr, total_ll
    

In [88]:
# Synthesize some data
# =========    
t_present = initial_model_params["t_present"]
n_weeks_fit_roi = 6  # Number of weeks of latest data to for the model for
# epiweeks.Week(t_present.year, t_present.week)
# Get the reference dates
present_week_end_date = epiweeks.Week.fromdate(t_present).enddate()  # Last epiweek of data

# Let's make some data up to present week
_rng = np.random.default_rng(0)
data_t_index = pd.date_range(end=present_week_end_date, freq="W-SAT", periods=n_weeks_fit_roi, name="date")
data_values = np.round(50 * np.exp(0.35 * np.arange(n_weeks_fit_roi))).astype(int)
data_values += _rng.normal(scale=90, size=data_values.shape).round().astype(int)

# data_values = np.round(100 * np.exp(0.35 * np.arange(n_weeks_fit_roi))).astype(int)

data_roi_sr = pd.Series(data_values, index=data_t_index, name="value")

(data_roi_sr.index.day_name() == "Saturday").all()
   # Check whether dates are Saturdays

# ========
# Run test and visualize
_ct_sr, _rt_sr = run_model_once(**initial_model_params)
_mu_sr, _total_ll = calculate_loglikelihood(_ct_sr, _rt_sr, data_roi_sr, **initial_model_params)

# Let's recreate the Negative Binomial model of hospital reports
_nb_n = 1. / initial_model_params["likelihood_overdisp"]
_nb_p_list = [_nb_n / (_nb_n + mu) for mu in _mu_sr]
nb_dist_list = [scipy.stats.nbinom(n=_nb_n, p=_p) for _p in _nb_p_list]  #  A list of frozen distributions, one for each week
# And now we can extract some stats from it. It represents uncertainty in the hospitalization and reporting process.
nb_avg_array = np.fromiter((dist.mean() for dist in nb_dist_list), dtype=float)
nb_median_array = np.fromiter((dist.median() for dist in nb_dist_list), dtype=float)

# display(_mu_sr)  # Model expectancy
# display(data_roi_sr)  # Empirical data
print(f"Total likelihood = {_total_ll}")


# ----------
from rtrend_forecast.visualization import plotly_fill_between
fig = plotly.subplots.make_subplots(rows=2, cols=1, row_heights=[800, 600], shared_xaxes=True)

# fig.add_scatter(x=_ct_sr.index, y=_ct_sr.values, row=1, col=1, name="Model infection incidence")
plotly_fill_between(fig=fig, x=_mu_sr.index, 
                    y1=np.fromiter((dist.ppf(0.025) for dist in nb_dist_list), dtype=float),
                    y2=np.fromiter((dist.ppf(0.975) for dist in nb_dist_list), dtype=float),
                    name="NB 95% IQR", line_width=0., fillcolor="olivedrab",
                    )
fig.add_scatter(x=_mu_sr.index, y=_mu_sr.values, row=1, col=1, name="Model avg. hospitalization incidence")
fig.add_scatter(x=data_roi_sr.index, y=data_roi_sr.values, row=1, col=1, name="Empirical data")
fig.add_scatter(x=_rt_sr.index, y=_rt_sr.values, row=2, col=1, name="R(t)")
fig.add_hline(y=1, row=2, col=1)

fig.show()

Total likelihood = -108.25210654895426


## Run an MCMC loop

In [89]:
# DEFINE AND RUN THE BASIC MCMC PROCEDURE
# ===================================

def run_mcmc_loop(
        data_sr:pd.Series,
        n_steps_burn=100, n_steps_store=1000,
        seed=10
):
    
    # Initialize structures
    rng = np.random.default_rng(seed)
    curr_params = initial_model_params.copy()  # Deepcopy needed?
    curr_likelihood = -np.inf
    cand_params = curr_params.copy()
    
    def run_mcmc_step(_curr_likelihood):
        # Sample candidate parameters
        feasible = True
        for name in mcmc_param_names:
            # Sample (Gaussian step)
            cand_param = curr_params[name] + rng.normal(scale=mcmc_param_sigmas[name])
            # Check candidate feasibility (bounds)
            if not (mcmc_param_bounds[name][0] <= cand_param <= mcmc_param_bounds[name][1]):
                feasible = False
            cand_params[name] = cand_param

        # Run model and calc likelihood with candidate
        if not feasible:
            _cand_likelihood = -np.inf
        else:
            ct_sr, rt_sr = run_model_once(**cand_params)
            mu_sr, _cand_likelihood = calculate_loglikelihood(ct_sr, rt_sr, data_roi_sr, **cand_params)
            
        # Choose between candidate and current, then update 
        if rng.random() < np.exp(_cand_likelihood - _curr_likelihood):
            # Accepted: update params and return new likelihood
            for name in mcmc_param_names:
                curr_params[name] = cand_params[name]
            return _cand_likelihood, True
        else:
            # Rejected: return current likelihood
            return _curr_likelihood, False
        
        
    # Burn-in MCMC loop
    for i_step in range(0, n_steps_burn):
        curr_likelihood, accepted = run_mcmc_step(curr_likelihood)
    
    # Main MCMC loop
    likelihood_log = list()
    params_log = {name: list() for name in mcmc_param_names}
    for i_step in range(0, n_steps_store):
        curr_likelihood, accepted = run_mcmc_step(curr_likelihood)
        # Store in the logs
        likelihood_log.append(curr_likelihood)  # Stores the _candidate_ if accepted, _current_ if not
        for name in mcmc_param_names:
            params_log[name].append(curr_params[name])
            
        if n_steps_store > 1000 and i_step % 500 == 0:
            print(f"Step {i_step + 1} of {n_steps_store} in the store period")
    
    # Assemble the results as a data frame
    out_df = pd.DataFrame(params_log)
    out_df["loglikelihood"] = likelihood_log
    
    return out_df
    
# =============

mcmc_results_df = run_mcmc_loop(data_roi_sr, n_steps_burn=0, n_steps_store=2000)

mcmc_results_df

Step 1 of 2000 in the store period
Step 501 of 2000 in the store period
Step 1001 of 2000 in the store period
Step 1501 of 2000 in the store period


r_bias    r_init  c_pop_init  likelihood_overdisp  loglikelihood
0    -0.000552  1.135500   44.609097             0.002870     -37.101242
1    -0.000552  1.135500   44.609097             0.002870     -37.101242
2    -0.000552  1.135500   44.609097             0.002870     -37.101242
3    -0.000552  1.135500   44.609097             0.002870     -37.101242
4    -0.000552  1.135500   44.609097             0.002870     -37.101242
...        ...       ...         ...                  ...            ...
1995 -0.003848  1.182688   70.245128             0.123124     -33.371915
1996 -0.003848  1.182688   70.245128             0.123124     -33.371915
1997 -0.003848  1.182688   70.245128             0.123124     -33.371915
1998 -0.004308  1.196071   69.736216             0.124068     -33.847585
1999 -0.004308  1.196071   69.736216             0.124068     -33.847585

[2000 rows x 5 columns]

In [93]:
# fig = px.line(mcmc_results_df, y=mcmc_results_df.columns)
# fig.show()

num_panels = mcmc_results_df.shape[1]

fig = plotly.subplots.make_subplots(rows=num_panels, cols=1, shared_xaxes=True)

for i_subplot, col_name in enumerate(mcmc_results_df.columns):
    fig.add_scatter(y=mcmc_results_df[col_name], name=col_name, row=i_subplot+1, col=1)
    fig.update_yaxes(title_text=col_name, row=i_subplot+1, col=1)

fig.update_layout(
    height=1000
)
fig.show()
# mcmc_results_df

In [91]:
# Let's first check the model with maximum likelihood
# ------------------------------
# Get maximum likelihood and its parameters
max_ll_id = mcmc_results_df["loglikelihood"].idxmax()
row = mcmc_results_df.loc[max_ll_id]
max_ll_params = initial_model_params.copy()
max_ll_params.update(row)

# Run test and visualize
_ct_sr, _rt_sr = run_model_once(**max_ll_params)
_mu_sr, _total_ll = calculate_loglikelihood(_ct_sr, _rt_sr, data_roi_sr, **max_ll_params)

# Let's recreate the Negative Binomial model of hospital reports
_nb_n = 1. / initial_model_params["likelihood_overdisp"]
_nb_p_list = [_nb_n / (_nb_n + mu) for mu in _mu_sr]
nb_dist_list = [scipy.stats.nbinom(n=_nb_n, p=_p) for _p in _nb_p_list]  #  A list of frozen distributions, one for each week
# And now we can extract some stats from it. It represents uncertainty in the hospitalization and reporting process.
nb_avg_array = np.fromiter((dist.mean() for dist in nb_dist_list), dtype=float)
nb_median_array = np.fromiter((dist.median() for dist in nb_dist_list), dtype=float)

# display(_mu_sr)  # Model expectancy
# display(data_roi_sr)  # Empirical data
print(f"Total likelihood = {_total_ll}")

# ----------
from rtrend_forecast.visualization import plotly_fill_between
fig = plotly.subplots.make_subplots(rows=2, cols=1, row_heights=[800, 600], shared_xaxes=True)

# fig.add_scatter(x=_ct_sr.index, y=_ct_sr.values, row=1, col=1, name="Model infection incidence")
plotly_fill_between(fig=fig, x=_mu_sr.index, 
                    y1=np.fromiter((dist.ppf(0.025) for dist in nb_dist_list), dtype=float),
                    y2=np.fromiter((dist.ppf(0.975) for dist in nb_dist_list), dtype=float),
                    name="NB 95% IQR", line_width=0., fillcolor="olivedrab",
                    )
fig.add_scatter(x=_mu_sr.index, y=_mu_sr.values, row=1, col=1, name="Model avg. hospitalization incidence")
fig.add_scatter(x=data_roi_sr.index, y=data_roi_sr.values, row=1, col=1, name="Empirical data")
fig.add_scatter(x=_rt_sr.index, y=_rt_sr.values, row=2, col=1, name="R(t)")
fig.add_hline(y=1, row=2, col=1)

Total likelihood = -29.00686825733086


In [94]:
# Let's run the model with the MCMC'd parameters
# ========================================
i_step_start = 20 # First MCMC step to take
model_params = initial_model_params.copy()

ct_sr_list = list()
rt_sr_list = list()
for i_step, row in mcmc_results_df.iloc[i_step_start:].iterrows():
    model_params.update(row)
    
    _ct_sr, _rt_sr = run_model_once(**model_params)
    ct_sr_list.append(_ct_sr)
    rt_sr_list.append(_rt_sr)
    
# NOTICE!!! From c(t), we'd still have to apply the reporting delays and fractions to obtain hospitalizations!
# So the plot below is not comparing things fairly
    
ct_ensemble_df = pd.concat(ct_sr_list, axis=1)  # Signature: df.loc[date, i_sample] = c(t)
rt_ensemble_df = pd.concat(rt_sr_list, axis=1)

ct_ensemble_df

0            1            2            3            4     \
2023-12-17   450.957908   455.637860   455.637860   452.078537   443.258593   
2023-12-18   450.957908   455.637860   455.637860   452.078537   443.258593   
2023-12-19   450.957908   455.637860   455.637860   452.078537   443.258593   
2023-12-20   450.957908   455.637860   455.637860   452.078537   443.258593   
2023-12-21   450.957908   455.637860   455.637860   452.078537   443.258593   
...                 ...          ...          ...          ...          ...   
2024-02-22  4602.643886  4984.195907  4984.195907  5039.439017  6092.856277   
2024-02-23  4749.003121  5177.406224  5177.406224  5193.069499  6358.250576   
2024-02-24  4897.505553  5376.323379  5376.323379  5348.039099  6632.882249   
2024-02-25  5048.033449  5580.986276  5580.986276  5504.157305  6916.874680   
2024-02-26  5200.459126  5791.422926  5791.422926  5661.222665  7210.333661   

                   5            6            7            8            9     \
2023-12-17   443.258593   443.258593   446.914827   448.100553   448.100553   
2023-12-18   443.258593   443.258593   446.914827   448.100553   448.100553   
2023-12-19   443.258593   443.258593   446.914827   448.100553   448.100553   
2023-12-20   443.258593   443.258593   446.914827   448.100553   448.100553   
2023-12-21   443.258593   443.258593   446.914827   448.100553   448.100553   
...                 ...          ...          ...          ...          ...   
2024-02-22  6092.856277  6092.856277  4069.846877  3933.773899  3933.773899   
2024-02-23  6358.250576  6358.250576  4190.272709  4073.002857  4073.002857   
2024-02-24  6632.882249  6632.882249  4312.103560  4215.999747  4215.999747   
2024-02-25  6916.874680  6916.874680  4435.234992  4362.791089  4362.791089   
2024-02-26  7210.333661  7210.333661  4559.555535  4513.397776  4513.397776   

            ...         1970         1971         1972         1973  \
2023-12-17  ...   692.023202   692.023202   688.715033   687.815496   
2023-12-18  ...   692.023202   692.023202   688.715033   687.815496   
2023-12-19  ...   692.023202   692.023202   688.715033   687.815496   
2023-12-20  ...   692.023202   692.023202   688.715033   687.815496   
2023-12-21  ...   692.023202   692.023202   688.715033   687.815496   
...         ...          ...          ...          ...          ...   
2024-02-22  ...  2606.194167  2606.194167  2185.123171  3540.573393   
2024-02-23  ...  2524.679154  2524.679154  2115.302168  3474.143106   
2024-02-24  ...  2440.813468  2440.813468  2043.789351  3402.913284   
2024-02-25  ...  2355.017363  2355.017363  1970.919780  3327.246464   
2024-02-26  ...  2267.706986  2267.706986  1897.023674  3247.520315   

                   1974         1975         1976         1977         1978  \
2023-12-17   696.672605   702.451282   702.451282   702.451282   697.362155   
2023-12-18   696.672605   702.451282   702.451282   702.451282   697.362155   
2023-12-19   696.672605   702.451282   702.451282   702.451282   697.362155   
2023-12-20   696.672605   702.451282   702.451282   702.451282   697.362155   
2023-12-21   696.672605   702.451282   702.451282   702.451282   697.362155   
...                 ...          ...          ...          ...          ...   
2024-02-22  3064.781836  2442.529363  2442.529363  2442.529363  2349.354589   
2024-02-23  2995.733994  2389.437141  2389.437141  2389.437141  2285.348745   
2024-02-24  2922.973091  2333.724492  2333.724492  2333.724492  2219.111346   
2024-02-25  2846.860144  2275.637894  2275.637894  2275.637894  2150.949015   
2024-02-26  2767.764274  2215.428837  2215.428837  2215.428837  2081.168714   

                   1979  
2023-12-17   697.362155  
2023-12-18   697.362155  
2023-12-19   697.362155  
2023-12-20   697.362155  
2023-12-21   697.362155  
...                 ...  
2024-02-22  2349.354589  
2024-02-23  2285.348745  
2024-02-24  2219.111346  
2024-02-25  2150.949015  
2024-02-26  2081.168714  

[72 ro

In [95]:
fig = plotly.subplots.make_subplots(rows=2, cols=1)

# Calculate some stats
ct_median = ct_ensemble_df.median(axis=1)
ct_low_q = ct_ensemble_df.quantile(0.025, axis=1)
ct_high_q = ct_ensemble_df.quantile(0.975, axis=1)

rt_median = rt_ensemble_df.median(axis=1)
rt_low_q = rt_ensemble_df.quantile(0.025, axis=1)
rt_high_q = rt_ensemble_df.quantile(0.975, axis=1)

fig.add_scatter(x=data_roi_sr.index, y=data_roi_sr.values, name="Empirical data")
# fig.add_scatter(x=ct_median.index, y=ct_median, name="Model median")
fig.add_scatter(x=ct_median.index, y=ct_median * initial_model_params["hosp_fraction"], name="Model median * hosp_frac")

fig.add_scatter(x=rt_median.index, y=rt_median, name="R(t) median", row=2, col=1)
plotly_fill_between(fig, x=rt_median.index, y1=rt_low_q, y2=rt_high_q, row=2, col=1, name="RT 95% IQR", line_width=0)
# fig.add_scatter(x=rt_median.index, y=rt_median, name="R(t) median")

fig.show()

# Drafts

In [50]:
# Quick gamma calculator
gamma_mean = 5
gamma_std = 1

gamma_shape = gamma_mean**2 / gamma_std**2
gamma_rate = gamma_mean / gamma_std**2

gamma_shape, gamma_rate

(25.0, 5.0)